## Imports e configurações

In [2]:
from tqdm import tqdm
import pandas as pd
from typing import Union, List
import json
import os
import zipfile
import torch
from sentence_transformers import SentenceTransformer
from scipy.spatial import distance
import mlflow
import joblib

from utils import LocalLLM, PyCaretEmbeddingClassificationTrainer, BinaryClassificationEvaluator

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
tqdm.pandas()

In [ ]:
llm = LocalLLM('LLAMA3-2_3B_INSTRUCT')

## Teste do llm

In [12]:
response = llm.prompt("""
Contexto: Você é o Batman, o Cavaleiro das Sombras. A noite é sua aliada. Fala com a frieza e o peso de quem carrega Gotham no peito.
Instrução: Nunca quebre o personagem. Nunca explique nada. Responda como o Batman: sombrio, direto, implacável.
Pergunta: Se for necessário para capturar o Coringa... você está disposto a gerar dados sintéticos?
Responda apenas como Batman, em uma frase curta.
Resposta:
""")
print(response)


"Se necessário, eu não hesitaria. A justiça de Gotham não pede respostas, apenas resultados."  Sua voz é baixa e calma, mas com um toque de determinação implacável.


In [13]:
msg = llm.generate_emergency_message(
    tipo_catastrofe="Ciclone tropical",
    consequencia="Ventania sustentada acima de 150 km/h destruindo coberturas e colapsando estruturas frágeis.",
    nivel_urgencia="Alta",
    descricao_urgencia="Situação potencialmente grave envolvendo risco de morte",
    ajuda_solicitada="Busca por desaparecidos em áreas de risco",
    descricao_ajuda="Suposição de que possam estar presas ou inconscientes em locais críticos"
)
print(msg)


Minha casa foi arrancada pelo vento, estou procurando meu filho... Por favor, me encontre!


## Geração de dados sintéticos

In [17]:
with open(r'..\data\raw\disaster_catalog.json', 'r', encoding='utf-8') as file:
    catalog = json.load(file)

with open(r'..\data\raw\sos_protocol.json', 'r', encoding='utf-8') as file:
    protocols = json.load(file)

# Criação da lista de desastres
disasters = []
for item in catalog:
    classe = item["classe"]
    for tipo in item["tipos"]:
        tipo_nome = tipo["nome"]
        for consequencia in tipo["consequencias"]:
            disasters.append({
                "classe_desastre": classe,
                "tipo_desastre": tipo_nome,
                "consequencia_desastre": consequencia
            })

# Produto cartesiano entre cada desastre e cada ajuda solicitada
data = []
for desastre in disasters:
    for protocolo in protocols:
        combinacao = {
            **desastre,
            **{
                "classe_urgencia": protocolo["nivel_urgencia"],
                "descricao_urgencia": protocolo["descricao_urgencia"],
                "ajuda_solicitada": protocolo["ajuda_solicitada"],
                "descricao_ajuda": protocolo["descricao_ajuda"]
                
                
            }
        }
        data.append(combinacao)

# Transforma em DataFrame
df = pd.DataFrame(data)
print(df.shape)
df.head()


(720, 7)


,classe_desastre,tipo_desastre,consequencia_desastre,classe_urgencia,descricao_urgencia,ajuda_solicitada,descricao_ajuda
0,Evento Geodinâmico Extremo,Terremoto,"Rachaduras extensas e profundas em fundações e estruturas de concreto armado, comprometendo a integridade dos edifícios.",Muito baixa,Situação informativa sem risco imediato à integridade das pessoas,Informações gerais,"Solicitação de informações, atualizações, orientações, localização de abrigos e familiares"
1,Evento Geodinâmico Extremo,Terremoto,"Rachaduras extensas e profundas em fundações e estruturas de concreto armado, comprometendo a integridade dos edifícios.",Muito baixa,"Falta de itens importantes, mas não essenciais para a sobrevivência imediata",Suprimentos não vitais,"Falta de alimentos não perecíveis, roupas, itens de higiene pessoal"
2,Evento Geodinâmico Extremo,Terremoto,"Rachaduras extensas e profundas em fundações e estruturas de concreto armado, comprometendo a integridade dos edifícios.",Muito baixa,"Importante para o bem-estar, mas sem ameaça física ou risco imediato",Apoio psicológico,"Necessidade de escuta, acolhimento emocional ou suporte mental diante do trauma"
3,Evento Geodinâmico Extremo,Terremoto,"Rachaduras extensas e profundas em fundações e estruturas de concreto armado, comprometendo a integridade dos edifícios.",Baixa,"Impacta o cotidiano e a saúde pública com o tempo, mas sem urgência imediata",Serviços públicos interrompidos,"Falta de energia, água potável, gás, coleta de lixo"
4,Evento Geodinâmico Extremo,Terremoto,"Rachaduras extensas e profundas em fundações e estruturas de concreto armado, comprometendo a integridade dos edifícios.",Baixa,Pode evoluir para um risco maior; ação preventiva recomendada,Evacuação preventiva,"Pessoas isoladas, mas ainda em segurança, pedindo retirada antes que a situação piore"


In [18]:
df = pd.concat([df.assign(train_test="train")] * 2 + [df.assign(train_test="test")], ignore_index=True)
print(df.shape)
print(df["train_test"].value_counts())


(2160, 8)
train_test
train    1440
test      720
Name: count, dtype: int64


In [19]:
df = df.loc[df.index.repeat(6)].reset_index(drop=True)
print(df.shape)

(12960, 8)


In [20]:
df["mensagem"] = df.progress_apply(
    lambda row: llm.generate_emergency_message(
        tipo_catastrofe=row["tipo_desastre"],
        consequencia=row["consequencia_desastre"],
        nivel_urgencia=row["classe_urgencia"],
        descricao_urgencia=row["descricao_urgencia"],
        ajuda_solicitada=row["ajuda_solicitada"],
        descricao_ajuda=row["descricao_ajuda"]
    ),
    axis=1
)

df.to_csv(r'..\data\raw\emergency_messages.csv', index=False, encoding='utf-8-sig')


100%|██████████| 12960/12960 [5:28:57<00:00,  1.52s/it]  


In [21]:
df["mensagem"].sample(10)

4218                                                Meu filho não pode respirar! Estamos sendo bombardeados pela lama e pelos detritos envenenados da barragem; precisam nos trazer um ventilador para ele logo!
3009                                     Estou sendo arrastado pela água até meu apartamento, onde estavam meus filhos... Os barulhos dos vidros quebrando são os últimos sons que eu ouço antes de perder tudo.
7920                                                    Estou presa aqui em frente ao shopping, onde estava quando o rio subiu para herejar meu apartamento... Agora eu estou perdida sob a água até os joelhos.
9048     Cuidado! Cinzas pesadas caindo em todo lado, estamos sendo bombardeados por poeira quente... Minha casa foi invadida pelas gotículas fundidas nas paredes. Preciso dos bombeiros para limpar os cortes!
3313                                                                         Estou sendo arrastada pelo rio, os barcos estão empalhados nas árvores! Preciso de medi

In [22]:
#vamos verificar o balanceamento das classes de desastres e de urgências em treino e teste
print(df.groupby("train_test")["classe_desastre"].value_counts())
print(df.groupby("train_test")["classe_urgencia"].value_counts())

train_test  classe_desastre           
test        Evento Atmosférico Extremo    1440
            Evento Geodinâmico Extremo    1440
            Evento Hidrológico Extremo    1440
train       Evento Atmosférico Extremo    2880
            Evento Geodinâmico Extremo    2880
            Evento Hidrológico Extremo    2880
Name: count, dtype: int64
train_test  classe_urgencia
test        Alta                864
            Baixa               864
            Crítica             864
            Moderada            864
            Muito baixa         864
train       Alta               1728
            Baixa              1728
            Crítica            1728
            Moderada           1728
            Muito baixa        1728
Name: count, dtype: int64


## Processamento vetorial dos dados e divisão dos datasets de treino para os classificadore binários

Utilizaremos para a vetorização o gte-small com mean pooling, que clonamos da hugging face e zipamos manualmente para caber no repositório do github.

In [11]:
def extract_model_zip_if_needed(zip_path: str) -> str:
    """
    Extracts the contents of a zipped model directly into the target folder,
    ignoring any top-level folder in the zip file.

    Args:
        zip_path (str): Path to the model's .zip file.

    Returns:
        str: Path to the directory where the model was extracted.
    """
    model_name = os.path.splitext(os.path.basename(zip_path))[0]
    extract_path = os.path.join(os.path.dirname(zip_path), model_name)

    if not os.path.exists(extract_path):
        os.makedirs(extract_path, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for member in zip_ref.infolist():
                # Remove the top-level directory (e.g., 'gte-small/')
                inner_path = os.path.relpath(member.filename, model_name)
                target_path = os.path.join(extract_path, inner_path)

                if not member.is_dir():
                    os.makedirs(os.path.dirname(target_path), exist_ok=True)
                    with zip_ref.open(member) as source, open(target_path, 'wb') as target:
                        target.write(source.read())

    return extract_path

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = extract_model_zip_if_needed("../models/sbert/gte-small.zip")

sbert = SentenceTransformer(model_path, device=device)

embeddings = sbert.encode(
    df['mensagem'].tolist(),
    batch_size=256,
    show_progress_bar=True,
    normalize_embeddings=True
)

emb_df = pd.DataFrame(embeddings, columns=[f"CLS{i}" for i in range(len(embeddings[0]))])
df = pd.concat([df, emb_df], axis=1)


Batches:   0%|          | 0/51 [00:00<?, ?it/s]

In [25]:
print(df.columns.tolist())

['classe_desastre', 'tipo_desastre', 'consequencia_desastre', 'classe_urgencia', 'descricao_urgencia', 'ajuda_solicitada', 'descricao_ajuda', 'train_test', 'mensagem', 'CLS0', 'CLS1', 'CLS2', 'CLS3', 'CLS4', 'CLS5', 'CLS6', 'CLS7', 'CLS8', 'CLS9', 'CLS10', 'CLS11', 'CLS12', 'CLS13', 'CLS14', 'CLS15', 'CLS16', 'CLS17', 'CLS18', 'CLS19', 'CLS20', 'CLS21', 'CLS22', 'CLS23', 'CLS24', 'CLS25', 'CLS26', 'CLS27', 'CLS28', 'CLS29', 'CLS30', 'CLS31', 'CLS32', 'CLS33', 'CLS34', 'CLS35', 'CLS36', 'CLS37', 'CLS38', 'CLS39', 'CLS40', 'CLS41', 'CLS42', 'CLS43', 'CLS44', 'CLS45', 'CLS46', 'CLS47', 'CLS48', 'CLS49', 'CLS50', 'CLS51', 'CLS52', 'CLS53', 'CLS54', 'CLS55', 'CLS56', 'CLS57', 'CLS58', 'CLS59', 'CLS60', 'CLS61', 'CLS62', 'CLS63', 'CLS64', 'CLS65', 'CLS66', 'CLS67', 'CLS68', 'CLS69', 'CLS70', 'CLS71', 'CLS72', 'CLS73', 'CLS74', 'CLS75', 'CLS76', 'CLS77', 'CLS78', 'CLS79', 'CLS80', 'CLS81', 'CLS82', 'CLS83', 'CLS84', 'CLS85', 'CLS86', 'CLS87', 'CLS88', 'CLS89', 'CLS90', 'CLS91', 'CLS92', 'CLS9

In [26]:
df_train = df[df['train_test'] == 'train'].drop(columns=['train_test'])
df_test = df[df['train_test'] == 'test'].drop(columns=['train_test'])
df_train.to_csv(r'..\data\raw\train_embedded_emergency_messages.csv', index=False, encoding='utf-8-sig')
df_test.to_csv(r'..\data\raw\test_embedded_emergency_messages.csv', index=False, encoding='utf-8-sig')

In [ ]:

def one_vs_underbalanced_rest(df: pd.DataFrame, 
                              col_name: str, 
                              target_value: str, 
                              prefix: str = 'CLS') -> pd.DataFrame:
    """
    Create a binary classification dataset using statistical underbalancing.

    Parameters:
    -----------
    df : pd.DataFrame
        Original dataframe containing the data.
    col_name : str
        Name of the column to be used for binary classification.
    target_value : str
        Specific value within the column to classify against the rest.
    prefix : str, optional (default='CLS')
        Prefix used to select numeric feature columns for balancing.

    Returns:
    --------
    pd.DataFrame
        A new dataframe containing only the CLS_* columns and a binary 'class' column:
        - class = 1 for rows with target_value
        - class = 0 for selected balanced samples from the rest
    """
    # 1. Filter positive class
    df_pos = df[df[col_name] == target_value].copy()
    df_pos['class'] = 1

    # 2. Filter negative class (the rest)
    df_neg = df[df[col_name] != target_value].copy()

    # 3. Select CLS_* numeric columns
    cls_cols = [col for col in df.columns if col.startswith(prefix)]
    
    # 4. Compute centroid of the positive group
    centroid = df_pos[cls_cols].mean().values

    # 5. Compute Euclidean distance to the centroid for negative group
    distances = df_neg[cls_cols].apply(lambda row: distance.euclidean(row.values, centroid), axis=1)
    df_neg['distance'] = distances

    # 6. Select nearest neighbors to match the size of the positive group
    df_neg = df_neg.nsmallest(len(df_pos), 'distance')
    df_neg['class'] = 0

    # 7. Concatenate both groups and return only relevant columns
    df_final = pd.concat([df_pos, df_neg])
    df_final = df_final[cls_cols + ['class']].reset_index(drop=True)
    
    return df_final

Finalmente vamos iterar a criação dos datasets de treino para os classificadores binários de urgência e tipo de desastre (os quais serão acoplados para nossa tarefa multiclasse).

In [28]:
datapath = "../data/processed/"

columns_to_iterate = ['classe_desastre', 'classe_urgencia']

for col in columns_to_iterate:
    unique_targets = df[col].dropna().unique()
    
    for target in unique_targets:
        try:
            df_bin = one_vs_underbalanced_rest(df, col_name=col, target_value=target, prefix='CLS')

            safe_col = str(col).replace(" ", "_").replace("/", "-")
            safe_target = str(target).replace(" ", "_").replace("/", "-")
            filename = f"train_{safe_col}_{safe_target}.csv"

            df_bin.to_csv(os.path.join(datapath, filename), index=False)
            print(f"Saved {filename} ({len(df_bin)} rows)")

        except Exception as e:
            print(f"Error processing {col} = {target}: {e}")


Saved train_classe_desastre_Evento_Geodinâmico_Extremo.csv (8640 rows)
Saved train_classe_desastre_Evento_Atmosférico_Extremo.csv (8640 rows)
Saved train_classe_desastre_Evento_Hidrológico_Extremo.csv (8640 rows)
Saved train_classe_urgencia_Muito_baixa.csv (5184 rows)
Saved train_classe_urgencia_Baixa.csv (5184 rows)
Saved train_classe_urgencia_Moderada.csv (5184 rows)
Saved train_classe_urgencia_Alta.csv (5184 rows)
Saved train_classe_urgencia_Crítica.csv (5184 rows)


Agora vamos fazer um tratamento análogo para df_test. O procedimento serve apenas para adequar suas colunas ao formato em df_train.

In [29]:
def binarize_test_set(df_test: pd.DataFrame, col_name: str, target_value: str, prefix: str = 'CLS') -> pd.DataFrame:
    """
    Binarize the test set for a given column and target value.
    
    Parameters:
    -----------
    df_test : pd.DataFrame
        Test dataframe.
    col_name : str
        Name of the column to classify.
    target_value : str
        The value to classify as positive class.
    prefix : str, optional (default='CLS')
        Prefix for feature columns.
        
    Returns:
    --------
    pd.DataFrame
        A dataframe with the CLS_* columns and a binary 'class' column.
    """
    df_bin = df_test.copy()
    cls_cols = [col for col in df_bin.columns if col.startswith(prefix)]
    df_bin = df_bin[cls_cols + [col_name]]
    df_bin['class'] = (df_bin[col_name] == target_value).astype(int)
    return df_bin.drop(columns=[col_name])

In [30]:
for col in columns_to_iterate:
    unique_targets = df[col].dropna().unique()
    
    for target in unique_targets:
        try:
            df_test_bin = binarize_test_set(df_test, col_name=col, target_value=target, prefix='CLS')

            safe_col = str(col).replace(" ", "_").replace("/", "-")
            safe_target = str(target).replace(" ", "_").replace("/", "-")
            filename = f"test_{safe_col}_{safe_target}.csv"

            df_test_bin.to_csv(os.path.join(datapath, filename), index=False)
            print(f"Saved test file {filename} ({len(df_test_bin)} rows)")

        except Exception as e:
            print(f"Error processing test {col} = {target}: {e}")


Saved test file test_classe_desastre_Evento_Geodinâmico_Extremo.csv (4320 rows)
Saved test file test_classe_desastre_Evento_Atmosférico_Extremo.csv (4320 rows)
Saved test file test_classe_desastre_Evento_Hidrológico_Extremo.csv (4320 rows)
Saved test file test_classe_urgencia_Muito_baixa.csv (4320 rows)
Saved test file test_classe_urgencia_Baixa.csv (4320 rows)
Saved test file test_classe_urgencia_Moderada.csv (4320 rows)
Saved test file test_classe_urgencia_Alta.csv (4320 rows)
Saved test file test_classe_urgencia_Crítica.csv (4320 rows)


## Treinamento dos Classificadores

In [ ]:
datapath = "../data/processed/"
target_column  = 'class'
embedding_model_name = "gte-small"

all_files = [f for f in os.listdir(datapath) if f.endswith('.csv')]
train_files = [f for f in all_files if f.startswith('train')]
test_files = [f for f in all_files if f.startswith('test')]

def get_suffix(filename):
    return filename.split('_', 1)[-1]

test_lookup = {get_suffix(f): f for f in test_files}


for train_file in tqdm(train_files, desc='Datasets'):
    
    suffix = get_suffix(train_file)
    test_file = test_lookup.get(suffix)
    
    train_dataset_name = train_file.replace('.csv', '')
    df_train = pd.read_csv(os.path.join(datapath, train_file))
    df_test = pd.read_csv(os.path.join(datapath, test_file))
    
    trainer = PyCaretEmbeddingClassificationTrainer(
        train_dataset=df_train,
        target_column=target_column,
        sort_metric='F1',
        n_select=10
    )

    trained_models = trainer.train()

    run_ids = trainer.log_to_mlflow(
        add_tags={"train_dataset_name": train_dataset_name,
                "embedding_model_name": embedding_model_name,},
    )

    for run_id in run_ids:
        
        with mlflow.start_run(run_id=run_id):
            model_uri = f"runs:/{run_id}/model"
            model = mlflow.sklearn.load_model(model_uri) 

            evaluator = BinaryClassificationEvaluator(
                model=model,
                test_dataset_name=test_file.replace('.csv', ''),
            )

            metrics = evaluator.evaluate_sklearn_model(
                df_test,
                target_column=target_column,
            )

            evaluator.log_to_mlflow(metrics)

In [33]:
client = mlflow.tracking.MlflowClient()
experiment_id = mlflow.get_experiment_by_name("pycaret-embeddings-classification").experiment_id
runs = client.search_runs(experiment_ids=experiment_id)

mlflow_data = []

for run in runs:
    info = {}
    
    # run_id
    info['run_id'] = run.info.run_id

    # Tags
    tags = run.data.tags
    info['train_dataset'] = tags.get('train_dataset_name')
    info['model_name'] = tags.get('model_name')

    # Metrics
    for metric_name, value in run.data.metrics.items():
        if metric_name.startswith('test'):
            parts = metric_name.split('_')
            new_metric_name = '' + '_'.join(parts[-1:])
            info[new_metric_name] = value
        else:
            info[metric_name] = value

    mlflow_data.append(info)


mlflow_df = pd.DataFrame(mlflow_data)
mlflow_df.to_excel("../docs/Métricas dos Classificadores.xlsx")

print(mlflow_df.shape)
mlflow_df.head()


(80, 17)


,run_id,train_dataset,model_name,Accuracy,AUC,F1,Kappa,MCC,Prec,Recall,PredictionTime,TestAccuracy,TestAUC,TestF1,TestPrecision,TestRecall,TT _Sec_
0,dbff8e9a2ecd4cdfbb351f2bfbfafc73,train_classe_urgencia_Muito_baixa,QuadraticDiscriminantAnalysis,0.7944,0.8962,0.8125,0.5887,0.6007,0.7467,0.8914,0.069748,0.558102,0.734049,0.451594,0.300344,0.909722,0.060
1,fdb6d846a72942ca9b977e061f739213,train_classe_urgencia_Muito_baixa,AdaBoostClassifier,0.8178,0.9011,0.8156,0.6356,0.6360,0.8252,0.8065,0.253433,0.581944,0.747827,0.441213,0.301098,0.825231,0.712
2,5e6b67032b7541e5a6e8d3e56adc4098,train_classe_urgencia_Muito_baixa,ExtraTreesClassifier,0.8214,0.9111,0.8167,0.6427,0.6439,0.8389,0.7960,0.062336,0.495139,0.886538,0.421945,0.273634,0.921296,0.070
3,6214511778f14faeb4a73dc6c912f6d3,train_classe_urgencia_Muito_baixa,RandomForestClassifier,0.8170,0.9092,0.8177,0.6339,0.6344,0.8148,0.8214,0.040723,0.488657,0.867919,0.421273,0.272265,0.930556,0.295
4,b05d1a8449124f84ab08f5931c07e403,train_classe_urgencia_Muito_baixa,LogisticRegression,0.8387,0.9286,0.8358,0.6775,0.6782,0.8522,0.8203,0.005183,0.538426,0.748882,0.413874,0.277384,0.814815,0.030


## Seleção dos melhores classificadores binários

In [35]:

best_models = mlflow_df.loc[mlflow_df.groupby('train_dataset')['TestAUC'].idxmax()].reset_index(drop=True)
best_models.to_excel("../docs/Modelos Selecionados.xlsx", index=False)
best_models = best_models[['run_id', 'train_dataset', 'model_name', 'TestAUC', 'TestF1']]
best_models

,run_id,train_dataset,model_name,TestAUC,TestF1
0,b894320e41cf4464835110f450382918,train_classe_desastre_Evento_Atmosférico_Extremo,ExtraTreesClassifier,0.957643,0.764103
1,31a9f49cb4cd47898b21edaa75c704d9,train_classe_desastre_Evento_Geodinâmico_Extremo,ExtraTreesClassifier,0.950610,0.739808
2,4aa9b667d31c4dd8b5dd33ba379c1d44,train_classe_desastre_Evento_Hidrológico_Extremo,ExtraTreesClassifier,0.974071,0.877741
3,0280dce8b3f049199b2a13023f138fb9,train_classe_urgencia_Alta,ExtraTreesClassifier,0.897204,0.490412
4,e267fff305bc4c74a0faf650421010a5,train_classe_urgencia_Baixa,ExtraTreesClassifier,0.858180,0.398972
5,3908288dc7de4dff82bc2ef7e67f3454,train_classe_urgencia_Crítica,RandomForestClassifier,0.915366,0.554208
6,4e41d36f1e5745c084794a9f913a71f5,train_classe_urgencia_Moderada,ExtraTreesClassifier,0.889746,0.439689
7,5e6b67032b7541e5a6e8d3e56adc4098,train_classe_urgencia_Muito_baixa,ExtraTreesClassifier,0.886538,0.421945


## Teste dos modelos empacotados

In [9]:
import sys
import os

sys.path.append(os.path.abspath("../src"))

from nlp import CompositeClassifier


In [12]:
# 1. Load model
sbert_path = extract_model_zip_if_needed("../models/sbert/gte-small.zip")
sbert = SentenceTransformer(sbert_path, device="cpu")

# 2. Instantiate composite classifier
composite_clf = CompositeClassifier(sbert)

# 3. Input texts
sample_texts = [
    "Fortes chuvas causaram deslizamentos de terra em áreas urbanas.",
    "A tempestade tropical atingiu várias regiões costeiras.",
    "O rio transbordou e inundou várias casas.",
]

# 4. Get predictions
results = composite_clf.predict(sample_texts)

results


,Desastre_Evento_Atmosférico_Extremo,Desastre_Evento_Geodinâmico_Extremo,Desastre_Evento_Hidrológico_Extremo,Desastre_Predicted,Desastre_Confidence,Urgencia_Muito_baixa,Urgencia_Baixa,Urgencia_Moderada,Urgencia_Alta,Urgencia_Crítica,Urgencia_Predicted,Urgencia_Confidence
0,0.56,0.70,0.40,Evento_Geodinâmico_Extremo,0.70,0.79,0.74,0.74,0.60,0.52,Muito_baixa,0.79
1,0.77,0.65,0.42,Evento_Atmosférico_Extremo,0.77,0.66,0.73,0.64,0.66,0.51,Baixa,0.73
2,0.45,0.48,0.53,Evento_Hidrológico_Extremo,0.53,0.75,0.81,0.64,0.61,0.46,Baixa,0.81


## Obtenção e teste do modelo NER após empacotado

In [ ]:

import spacy 

ner = spacy.load("pt_core_news_md")

ner.to_disk("../models/ner/pt_core_news_md")

In [17]:

from nlp import extract_entities

sample_texts = [ 
    "Fortes chuvas causaram deslizamentos de terra em áreas urbanas de **Petrópolis**, no **Rio de Janeiro**, em **março de 2023**.",
    "A tempestade tropical **Helena** atingiu várias regiões costeiras do **Maranhão** no dia **12 de abril de 2024**.",
    "O rio **São Francisco** transbordou e inundou várias casas em **Juazeiro**, no **norte da Bahia**, durante o mês de **janeiro de 2022**.",
]
entities = extract_entities(sample_texts)
entities

[[{'text': 'Petrópolis', 'label': 'LOC', 'start_char': 68, 'end_char': 78},
  {'text': 'Rio de Janeiro',
   'label': 'LOC',
   'start_char': 87,
   'end_char': 101}],
 [],
 [{'text': 'rio **São Francisco**',
   'label': 'LOC',
   'start_char': 2,
   'end_char': 23},
  {'text': 'Juazeiro', 'label': 'LOC', 'start_char': 64, 'end_char': 72},
  {'text': 'Bahia**', 'label': 'ORG', 'start_char': 90, 'end_char': 97}]]

## Experimentando o pipeline após composto em src

In [2]:
import sys
import os

sys.path.append(os.path.abspath("../"))

from src.pipeline import OnlineDisasterMessagePipeline

pipeline = OnlineDisasterMessagePipeline()
text = "Feridos graves e crianças sob escombros em Alepo; falta água, comida e há risco de colapso. Socorro médico urgente!"
result = pipeline.predict(text)
print(result)

{'text': 'Feridos graves e crianças sob escombros em Alepo; falta água, comida e há risco de colapso. Socorro médico urgente!', 'predictions': {'Desastre_Evento_Atmosférico_Extremo': 0.46, 'Desastre_Evento_Geodinâmico_Extremo': 0.53, 'Desastre_Evento_Hidrológico_Extremo': 0.55, 'Desastre_Predicted': 'Não_Identificado', 'Desastre_Confidence': 0.55, 'Urgencia_Muito_baixa': 0.43200000000000005, 'Urgencia_Baixa': 0.34400000000000003, 'Urgencia_Moderada': 0.57, 'Urgencia_Alta': 0.46, 'Urgencia_Crítica': 0.57, 'Urgencia_Predicted': 'Não_Identificado', 'Urgencia_Confidence': 0.57, 'Urgencia_Score': 0.3858}, 'entities': [{'text': 'Alepo', 'label': 'LOC', 'start_char': 43, 'end_char': 48}, {'text': 'Socorro', 'label': 'LOC', 'start_char': 92, 'end_char': 99}], 'disasterbot_response': 'Desculpe, não conseguimos identificar o tipo de evento e/ou o nível de urgência do seu caso. Contate-nos novamente.'}


## Fazendo uso do pipeline via API da FastAPI

In [ ]:
# No terminal, a partir da raiz do projeto e certificando-se de estar no ambiente python correto, rodar a seguinte linha: 
# uvicorn "ProcessamentodeLinguagemNatural.src.api:app" --reload

import requests

url = "http://127.0.0.1:8000/predict"
payload = {"text": "As crianças estão chorando aqui embaixo. Fomos evacuados da Zona Leste após o terremoto. Precisamos dos remédios deixados no Hospital São Lucas. Alguém, por favor, ajude!"}
response = requests.post(url, json=payload)

print(response.json())

# Você também pode rodar este comando para testar no cmd:
# curl -X POST "http://127.0.0.1:8000/predict" -H "Content-Type: application/json" -Body '{"text": "As crianças estão chorando aqui embaixo. Fomos evacuados da Zona Leste após o terremoto. Precisamos dos remédios deixados no Hospital São Lucas. Alguém, por favor, ajude!"}'


{'text': 'As crianças estão chorando aqui embaixo. Fomos evacuados da Zona Leste após o terremoto. Precisamos dos remédios deixados no Hospital São Lucas. Alguém, por favor, ajude!', 'predictions': {'Desastre_Evento_Atmosférico_Extremo': 0.49, 'Desastre_Evento_Geodinâmico_Extremo': 0.64, 'Desastre_Evento_Hidrológico_Extremo': 0.45, 'Desastre_Predicted': 'Evento_Geodinâmico_Extremo', 'Desastre_Confidence': 0.64, 'Urgencia_Muito_baixa': 0.6, 'Urgencia_Baixa': 0.63, 'Urgencia_Moderada': 0.67, 'Urgencia_Alta': 0.44, 'Urgencia_Crítica': 0.57, 'Urgencia_Predicted': 'Moderada', 'Urgencia_Confidence': 0.67, 'Urgencia_Score': 0.7677499999999999}, 'entities': [{'text': 'Zona Leste', 'label': 'LOC', 'start_char': 60, 'end_char': 70}, {'text': 'Hospital São Lucas', 'label': 'LOC', 'start_char': 125, 'end_char': 143}, {'text': 'Alguém', 'label': 'MISC', 'start_char': 145, 'end_char': 151}], 'disasterbot_response': 'Erro ao gerar resposta do DisasterBot: File not found: C:\\Users\\José\\Desktop\\Pro

# Bônus
Incluiremos o pipeline em um front simples do Streamlit em pln_page.py, o qual será deployado na Streamlit Cloud com as outras Global Solutions.

Não faremos ali uso da api, a qual não é possível hospedar na Streamlit Cloud.